## 1.시리즈와 데이터프레임에 apply메서드 사용하기

In [4]:
import pandas as pd
df = pd.DataFrame({
    'a': [10,20,30]
    ,'b': [20,30,40]
})
df

,a,b
0,10,20
1,20,30
2,30,40


In [10]:
df['a']**2

0    100
1    400
2    900
Name: a, dtype: int64

In [13]:
def my_exp(x, n):
    return x ** n
def my_sq(x):
    return x**2

print(df['a'].apply(my_sq))
print(df['a'].apply(lambda x:x**2))

0    100
1    400
2    900
Name: a, dtype: int64
0    100
1    400
2    900
Name: a, dtype: int64


In [15]:
# apply메서드에 첫 번째 인자에 n 제곱함수(my_exp_)를 전달하여 n 제곱 함수를 적용
df['a'].apply(my_exp, n=2)
df['a'].apply(my_exp, n=3)

0     1000
1     8000
2    27000
Name: a, dtype: int64

## 2. 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기

### 2-1. 데이터프레임의 누락값 처리하기 ― 열 방향

In [18]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.shape # (891,15)
titanic.head(4)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False


In [21]:
print(titanic.info()) # 891개가 아니면 누락값이 있다는 것

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [23]:
import numpy as np

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [25]:
# prop_missing 누락값의 비율 계산
# count_missing 함수를 이용하여 누락값을 계산
# size속성을 이용하여 df전체 데이터 수를 구하여 나눔
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    
    return num / dem
pmis_col = titanic.apply(prop_missing)
pmis_col

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64

In [28]:
# 누락값이 아닌 데이터 비율을 구함
# 전체 - 비율
def prop_complete(vec):
    return 1 - prop_missing(vec)
titanic.apply(prop_complete)

survived       1.000000
pclass         1.000000
sex            1.000000
age            0.801347
sibsp          1.000000
parch          1.000000
fare           1.000000
embarked       0.997755
class          1.000000
who            1.000000
adult_male     1.000000
deck           0.227834
embark_town    0.997755
alive          1.000000
alone          1.000000
dtype: float64

### 2-2. 데이터프레임의 누락값을 처리하기 ― 행 방뱡

In [33]:
# 행마다 누락값이 몇 개 있는가
titanic.apply(count_missing, axis=1) # 행마다 적용

0      1
1      0
2      1
3      0
4      1
      ..
886    1
887    0
888    2
889    0
890    1
Length: 891, dtype: int64

In [35]:
titanic.apply(count_missing, axis=0) # 열마다 적용

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [36]:
cmis_row = titanic.apply(count_missing, axis=1) # 누락값이 몇개있는가
pmis_row = titanic.apply(prop_missing, axis=1) # 누락값이 아닌 데이터비율
pcom_row = titanic.apply(prop_complete, axis=1) # 누락값비율

print(cmis_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64


In [41]:
titanic['num_missing'] = titanic.apply(count_missing, axis=1)
titanic.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,num_missing
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,1
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,1


In [47]:
# 누락값이 2개 이상인 데이터만 따로 모아옴
titanic.loc[titanic.num_missing > 1, :].sample(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,num_missing
511,0,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,2
773,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True,2
335,0,3,male,NaN,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True,2
42,0,3,male,NaN,0,0,7.8958,C,Third,man,True,NaN,Cherbourg,no,True,2
250,0,3,male,NaN,0,0,7.2500,S,Third,man,True,NaN,Southampton,no,True,2
828,1,3,male,NaN,0,0,7.7500,Q,Third,man,True,NaN,Queenstown,yes,True,2
837,0,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,2
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True,2
466,0,2,male,NaN,0,0,0.0000,S,Second,man,True,NaN,Southampton,no,True,2
611,0,3,male,NaN,0,0,7.0500,S,Third,man,True,NaN,Southampton,no,True,2


## 3. 연습문제

In [48]:
sesac= [{'name': 'A', 'birth': '1999-06-27', 'mid': 95, 'fin': 85},
             {'name': 'B', 'birth': '1997-06-27', 'mid': 85, 'fin': 80},
             {'name': 'C', 'birth': '1998-06-27', 'mid': 10, 'fin': 30},
             {'name': 'D', 'birth': '2000-06-27', 'mid': 73, 'fin': 90}]

df = pd.DataFrame(sesac, columns = ['name', 'birth', 'mid', 'fin'])
df

,name,birth,mid,fin
0,A,1999-06-27,95,85
1,B,1997-06-27,85,80
2,C,1998-06-27,10,30
3,D,2000-06-27,73,90


In [53]:
# 타입확인
df.dtypes

name      object
birth     object
mid        int64
fin        int64
tot        int64
avg      float64
dtype: object

In [51]:
#1
df['tot'] = df['mid'] + df['fin']
#2
df['avg'] = (df['mid'] + df['fin']) / 2
df

,name,birth,mid,fin,tot,avg
0,A,1999-06-27,95,85,180,90.0
1,B,1997-06-27,85,80,165,82.5
2,C,1998-06-27,10,30,40,20.0
3,D,2000-06-27,73,90,163,81.5


In [64]:
#3
def grade(v):
    if v >= 90:
        return 'A'
    elif v >= 80:
        return 'B'
    elif v >= 80:
        return 'C'
    else:
        return 'F'
df['grade'] = df['avg'].apply(grade)      
df

,name,birth,mid,fin,tot,avg,grade
0,A,1999-06-27,95,85,180,90.0,A
1,B,1997-06-27,85,80,165,82.5,B
2,C,1998-06-27,10,30,40,20.0,F
3,D,2000-06-27,73,90,163,81.5,B


In [69]:
#4
def pass_exam(v):
    if v in ('A','B','C'):
        return '합격'
    else:
        return '불합격'
df['grade1'] = df['grade'].apply(pass_exam)
df

,name,birth,mid,fin,tot,avg,grade,grade1
0,A,1999-06-27,95,85,180,90.0,A,합격
1,B,1997-06-27,85,80,165,82.5,B,합격
2,C,1998-06-27,10,30,40,20.0,F,불합격
3,D,2000-06-27,73,90,163,81.5,B,합격


In [75]:
#5
def birth_year(v):
    return int(v.split('-')[0])
df['year'] = df['birth'].apply(birth_year)
df

,name,birth,mid,fin,tot,avg,grade,grade1,year
0,A,1999-06-27,95,85,180,90.0,A,합격,1999
1,B,1997-06-27,85,80,165,82.5,B,합격,1997
2,C,1998-06-27,10,30,40,20.0,F,불합격,1998
3,D,2000-06-27,73,90,163,81.5,B,합격,2000


In [89]:
#6
from datetime import datetime
now = datetime.now() # datetime.datetime(2022, 7, 20, 16, 8, 0, 187244)
now.year # 2022

def age(v):
    return now.year - v + 1 # 태어날 때 1살
df['age'] = df['year'].apply(age)
df

,name,birth,mid,fin,tot,avg,grade,grade1,year,age
0,A,1999-06-27,95,85,180,90.0,A,합격,1999,24
1,B,1997-06-27,85,80,165,82.5,B,합격,1997,26
2,C,1998-06-27,10,30,40,20.0,F,불합격,1998,25
3,D,2000-06-27,73,90,163,81.5,B,합격,2000,23


In [142]:
#7
df[df['age'] >= 22]
#8
import pandas as pd
pd.merge(df[df['age'] >= 22], df[df['name']=='A'])
df.loc[(df['age'] >= 22) & (df['name']=='A')]

,name,birth,mid,fin,tot,avg,grade,grade1,year,age
0,A,1999-06-27,95,85,180,90.0,A,합격,1999,24


In [115]:
#9
df.loc[:,'birth':'grade']

,birth,mid,fin,tot,avg,grade
0,1999-06-27,95,85,180,90.0,A
1,1997-06-27,85,80,165,82.5,B
2,1998-06-27,10,30,40,20.0,F
3,2000-06-27,73,90,163,81.5,B


In [119]:
#10
df.loc[:,['name','grade']]

,name,grade
0,A,A
1,B,B
2,C,F
3,D,B


In [154]:
#11
s = []
for c in df.columns:
    if 'i' in c:
        s.append(c)
df[s]

df.loc[:,map(lambda x: 'i' in x,df.columns)]

df.filter(regex='i')

,birth,mid,fin
0,1999-06-27,95,85
1,1997-06-27,85,80
2,1998-06-27,10,30
3,2000-06-27,73,90
